# Análisis de Anomalías con Ingeniería de Características Avanzada

Sistema fotovoltaico con 24 inversores + datos ambientales e irradiancia
**Métodos**: Isolation Forest, Mahalanobis, LOF + Feature Engineering

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from scipy import stats
from scipy.signal import savgol_filter
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Carga y Unión de Datos

In [ ]:
# Cargar datos
env_data = pd.read_csv('environment_data.csv')
irr_data = pd.read_csv('irradiance_data.csv')
elec_data = pd.read_csv('electrical_data.csv')

# Convertir fechas
for df in [env_data, irr_data, elec_data]:
    df['measured_on'] = pd.to_datetime(df['measured_on'])

# Unir datos
data = env_data.merge(irr_data, on='measured_on', how='inner')
data = data.merge(elec_data, on='measured_on', how='inner')
data = data.sort_values('measured_on').reset_index(drop=True)

print(f"Datos unidos: {data.shape}")

## 2. Ingeniería de Características Avanzada

In [ ]:
# === CARACTERÍSTICAS TEMPORALES ===
data['hour'] = data['measured_on'].dt.hour
data['day_of_week'] = data['measured_on'].dt.dayofweek
data['month'] = data['measured_on'].dt.month
data['quarter'] = data['measured_on'].dt.quarter
data['is_weekend'] = (data['day_of_week'] >= 5).astype(int)

# Características cíclicas
data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)
data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)
data['day_sin'] = np.sin(2 * np.pi * data['day_of_week'] / 7)
data['day_cos'] = np.cos(2 * np.pi * data['day_of_week'] / 7)
data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

print("✓ Características temporales creadas")

In [ ]:
# === CARACTERÍSTICAS DE POTENCIA ===
# Identificar columnas de potencia AC
ac_cols = [col for col in data.columns if 'ac_power' in col]
data['total_ac_power'] = data[ac_cols].sum(axis=1)
data['mean_ac_power'] = data[ac_cols].mean(axis=1)
data['std_ac_power'] = data[ac_cols].std(axis=1)
data['cv_ac_power'] = data['std_ac_power'] / (data['mean_ac_power'] + 1e-8)

# Características de distribución
data['min_ac_power'] = data[ac_cols].min(axis=1)
data['max_ac_power'] = data[ac_cols].max(axis=1)
data['range_ac_power'] = data['max_ac_power'] - data['min_ac_power']
data['skew_ac_power'] = data[ac_cols].skew(axis=1)

# Percentiles
data['q25_ac_power'] = data[ac_cols].quantile(0.25, axis=1)
data['q75_ac_power'] = data[ac_cols].quantile(0.75, axis=1)
data['iqr_ac_power'] = data['q75_ac_power'] - data['q25_ac_power']

print("✓ Características de potencia creadas")

In [ ]:
# === CARACTERÍSTICAS DE EFICIENCIA ===
# Eficiencia teórica vs real
temp_col = 'ambient_temperature_o_149575'
irr_col = 'poa_irradiance_o_149574'
wind_col = 'wind_speed_o_149576'

# Potencia teórica esperada (modelo simplificado)
data['expected_power'] = np.where(
    data[irr_col] > 100,
    data[irr_col] * 0.8 * (1 - 0.004 * (data[temp_col] - 25)),
    0
)

# Ratios de eficiencia
data['efficiency_ratio'] = data['total_ac_power'] / (data['expected_power'] + 1e-8)
data['power_per_irradiance'] = data['total_ac_power'] / (data[irr_col] + 1e-8)

# Efectos ambientales
data['temp_effect'] = 1 - 0.004 * (data[temp_col] - 25)
data['wind_cooling'] = np.log1p(data[wind_col])  # Efecto de enfriamiento

print("✓ Características de eficiencia creadas")

In [ ]:
# === CARACTERÍSTICAS DE VENTANAS DESLIZANTES ===
# Funciones de ventana
def rolling_features(series, window=5):
    return pd.DataFrame({
        f'{series.name}_roll_mean': series.rolling(window, center=True).mean(),
        f'{series.name}_roll_std': series.rolling(window, center=True).std(),
        f'{series.name}_roll_min': series.rolling(window, center=True).min(),
        f'{series.name}_roll_max': series.rolling(window, center=True).max()
    })

# Aplicar a variables clave
key_vars = ['total_ac_power', temp_col, irr_col]
for var in key_vars:
    roll_feat = rolling_features(data[var])
    data = pd.concat([data, roll_feat], axis=1)

# Características de diferencias
data['power_diff'] = data['total_ac_power'].diff()
data['temp_diff'] = data[temp_col].diff()
data['irr_diff'] = data[irr_col].diff()

# Suavizado
if len(data) > 11:
    data['power_smooth'] = savgol_filter(data['total_ac_power'], 11, 3)
    data['power_residual'] = data['total_ac_power'] - data['power_smooth']

print("✓ Características de ventanas deslizantes creadas")

In [ ]:
# === CARACTERÍSTICAS DE INVERSORES INDIVIDUALES ===
# Número de inversores activos
data['active_inverters'] = (data[ac_cols] > 10).sum(axis=1)

# Desviación de cada inversor respecto al promedio
for i, col in enumerate(ac_cols[:5]):  # Solo primeros 5 para economizar espacio
    data[f'inv_{i+1}_deviation'] = data[col] - data['mean_ac_power']
    data[f'inv_{i+1}_zscore'] = stats.zscore(data[col])

print("✓ Características de inversores individuales creadas")

In [ ]:
# === CARACTERÍSTICAS DE INTERACCIÓN ===
# Interacciones importantes
data['temp_irr_interaction'] = data[temp_col] * data[irr_col]
data['wind_temp_interaction'] = data[wind_col] * data[temp_col]
data['power_efficiency_interaction'] = data['total_ac_power'] * data['efficiency_ratio']

# Ratios importantes
data['power_to_temp_ratio'] = data['total_ac_power'] / (data[temp_col] + 1e-8)
data['irr_to_wind_ratio'] = data[irr_col] / (data[wind_col] + 1e-8)

print("✓ Características de interacción creadas")

In [ ]:
# === CARACTERÍSTICAS DE ANOMALÍAS LOCALES ===
# Z-scores para detección de outliers
numerical_cols = data.select_dtypes(include=[np.number]).columns
for col in ['total_ac_power', 'efficiency_ratio', 'cv_ac_power']:
    if col in data.columns:
        data[f'{col}_zscore'] = np.abs(stats.zscore(data[col]))

# Distancia desde percentiles
data['power_from_median'] = np.abs(data['total_ac_power'] - data['total_ac_power'].median())
data['power_percentile'] = data['total_ac_power'].rank(pct=True)

print("✓ Características de anomalías locales creadas")

## 3. Selección y Preparación de Características

In [ ]:
# Seleccionar características para el modelo
feature_cols = [
    # Temporales
    'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos',
    'is_weekend',
    
    # Ambientales
    temp_col, irr_col, wind_col,
    
    # Potencia
    'total_ac_power', 'mean_ac_power', 'std_ac_power', 'cv_ac_power',
    'range_ac_power', 'skew_ac_power', 'active_inverters',
    
    # Eficiencia
    'efficiency_ratio', 'power_per_irradiance', 'temp_effect',
    
    # Ventanas deslizantes
    'total_ac_power_roll_mean', 'total_ac_power_roll_std',
    'power_diff', 'power_residual',
    
    # Interacciones
    'temp_irr_interaction', 'power_efficiency_interaction',
    
    # Anomalías locales
    'total_ac_power_zscore', 'power_from_median', 'power_percentile'
]

# Filtrar características que existen
feature_cols = [col for col in feature_cols if col in data.columns]

# Preparar matriz de características
X = data[feature_cols].copy()
X = X.fillna(X.median())  # Imputar con mediana

# Remover infinitos
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.median())

print(f"Características seleccionadas: {X.shape[1]}")
print(f"Datos preparados: {X.shape}")

## 4. Escalado Robusto y Detección de Anomalías

In [ ]:
# Escalado robusto (menos sensible a outliers)
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# === MÉTODO 1: ISOLATION FOREST ===
iso_forest = IsolationForest(contamination=0.05, random_state=42, n_estimators=200)
data['anomaly_iso'] = iso_forest.fit_predict(X_scaled)
data['score_iso'] = iso_forest.score_samples(X_scaled)

# === MÉTODO 2: MAHALANOBIS DISTANCE ===
elliptic = EllipticEnvelope(contamination=0.05, random_state=42)
data['anomaly_maha'] = elliptic.fit_predict(X_scaled)
data['score_maha'] = elliptic.mahalanobis(X_scaled)

# === MÉTODO 3: LOCAL OUTLIER FACTOR ===
lof = LocalOutlierFactor(contamination=0.05, n_neighbors=30)
data['anomaly_lof'] = lof.fit_predict(X_scaled)
data['score_lof'] = lof.negative_outlier_factor_

# Contar anomalías
methods = ['anomaly_iso', 'anomaly_maha', 'anomaly_lof']
for method in methods:
    count = (data[method] == -1).sum()
    print(f"{method}: {count} anomalías ({count/len(data)*100:.2f}%)")

## 5. Análisis de Consenso y Severidad

In [ ]:
# Consenso entre métodos
data['consensus_count'] = sum((data[method] == -1).astype(int) for method in methods)

# Niveles de severidad
data['severity'] = data['consensus_count'].map({
    0: 'Normal', 1: 'Bajo', 2: 'Medio', 3: 'Alto'
})

# Score combinado (promedio normalizado)
data['combined_score'] = (
    -data['score_iso'] +  # Isolation Forest (más negativo = más anómalo)
    data['score_maha'] +  # Mahalanobis (más positivo = más anómalo)
    -data['score_lof']    # LOF (más negativo = más anómalo)
) / 3

print("=== DISTRIBUCIÓN DE SEVERIDAD ===")
print(data['severity'].value_counts())
print(f"\nTotal anomalías: {(data['consensus_count'] > 0).sum()} ({(data['consensus_count'] > 0).sum()/len(data)*100:.2f}%)")

## 6. Visualización de Resultados

In [ ]:
# Visualización compacta
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Distribución de severidad
severity_counts = data['severity'].value_counts()
axes[0, 0].pie(severity_counts.values, labels=severity_counts.index, autopct='%1.1f%%')
axes[0, 0].set_title('Distribución de Severidad')

# 2. Score combinado vs eficiencia
normal = data[data['severity'] == 'Normal']
anomaly = data[data['severity'] != 'Normal']
axes[0, 1].scatter(normal['efficiency_ratio'], normal['combined_score'], 
                   alpha=0.5, s=10, label='Normal')
axes[0, 1].scatter(anomaly['efficiency_ratio'], anomaly['combined_score'], 
                   c='red', s=20, label='Anomalía')
axes[0, 1].set_xlabel('Ratio de Eficiencia')
axes[0, 1].set_ylabel('Score Combinado')
axes[0, 1].set_title('Eficiencia vs Score Combinado')
axes[0, 1].legend()

# 3. Importancia de características (aproximada)
feature_importance = np.abs(X_scaled.std(axis=0))
top_features = np.argsort(feature_importance)[-10:]
axes[1, 0].barh(range(len(top_features)), feature_importance[top_features])
axes[1, 0].set_yticks(range(len(top_features)))
axes[1, 0].set_yticklabels([feature_cols[i] for i in top_features])
axes[1, 0].set_title('Top 10 Características (por Varianza)')

# 4. Serie temporal de anomalías
sample_data = data.iloc[:2000]  # Muestra
axes[1, 1].plot(sample_data['measured_on'], sample_data['total_ac_power'], 
                'b-', alpha=0.6, linewidth=0.5)
high_anomalies = sample_data[sample_data['severity'] == 'Alto']
axes[1, 1].scatter(high_anomalies['measured_on'], high_anomalies['total_ac_power'], 
                   c='red', s=30, label='Alto Riesgo')
axes[1, 1].set_xlabel('Fecha')
axes[1, 1].set_ylabel('Potencia Total (W)')
axes[1, 1].set_title('Serie Temporal con Anomalías')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 7. Análisis de Características Más Importantes

In [ ]:
# Analizar qué características son más predictivas
anomaly_mask = data['consensus_count'] > 0

print("=== CARACTERÍSTICAS MÁS DISCRIMINATIVAS ===")
for col in ['cv_ac_power', 'efficiency_ratio', 'total_ac_power_zscore', 'power_residual']:
    if col in data.columns:
        normal_mean = data[~anomaly_mask][col].mean()
        anomaly_mean = data[anomaly_mask][col].mean()
        diff = abs(anomaly_mean - normal_mean)
        print(f"{col}: Normal={normal_mean:.3f}, Anomalía={anomaly_mean:.3f}, Diff={diff:.3f}")

# Exportar reporte final
report = data[data['consensus_count'] > 0][[
    'measured_on', 'total_ac_power', 'efficiency_ratio', 'cv_ac_power',
    'consensus_count', 'severity', 'combined_score'
]].copy()

report.to_csv('enhanced_anomaly_report.csv', index=False)
print(f"\n✓ Reporte mejorado guardado: {len(report)} anomalías")

## Conclusiones del Análisis Mejorado

### Mejoras Implementadas:
1. **Características Temporales Cíclicas**: Capturan patrones estacionales
2. **Métricas de Distribución**: CV, skewness, percentiles de inversores
3. **Ventanas Deslizantes**: Detectan cambios temporales
4. **Características de Eficiencia**: Comparan performance real vs esperada
5. **Interacciones**: Capturan relaciones complejas entre variables
6. **Escalado Robusto**: Menos sensible a outliers extremos

### Beneficios:
- **Mayor Precisión**: Más características = mejor detección
- **Menor Ruido**: Escalado robusto reduce falsos positivos
- **Interpretabilidad**: Score combinado facilita priorización
- **Contexto Operacional**: Características de eficiencia son accionables

### Próximos Pasos:
1. Validar con datos históricos de fallas conocidas
2. Implementar alertas por niveles de severidad
3. Crear dashboard interactivo
4. Entrenar modelo supervisado con etiquetas de mantenimiento